我们这里给三缺陷打上伪标签

In [1]:
import sys
sys.path.append('..')

In [2]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from py_file.func_Test import Test_Func
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch

In [3]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

# 加载数据和模型

## 获取数据

In [4]:
train_data_path = '/mnt/workspace/DATA/train_WM.npz'
train_data = np.load(train_data_path)
print(train_data.files)

['original_wm', 'label_one_hot', 'denoise_wm', 'label_name']


In [5]:
ori_wm = train_data['original_wm']
wm = train_data['denoise_wm']
label = train_data['label_name']
label_oh = train_data['label_one_hot']
three_ori_wm = []
three_wm = []
three_label = []
three_label_oh = []
for i in range(len(label_oh)):
    if label_oh[i].sum() == 3:
        three_ori_wm.append(np.array(ori_wm[i]))
        three_wm.append(np.array(wm[i]))  # 元素是tensor的列表无法转换为张量，所以这里把列表里的元素转换为np.array
        three_label.append(label[i])
        three_label_oh.append(label_oh[i])

In [6]:
three_wm_tensor = trans(torch.tensor(three_wm, dtype=torch.float32))
three_wm_tensor = torch.reshape(three_wm_tensor,(-1,1,224,224))  # 给图片加上通道
print(three_wm_tensor.shape)

/tmp/ipykernel_16264/2247876225.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  three_wm_tensor = trans(torch.tensor(three_wm, dtype=torch.float32))


torch.Size([8400, 1, 224, 224])


## 获取类别名称和多热编码对应的字典

In [7]:
three_label_unique, label_indices = np.unique(three_label, return_index=True)
print(three_label_unique)
print(label_indices)
# print(two_label_unique[indices])
three_label_oh_unique ,label_oh_indices = np.unique(three_label_oh, axis=0, return_index=True)
print(three_label_oh_unique)
print(label_oh_indices)

['C+EL+L' 'C+EL+S' 'C+ER+L' 'C+ER+S' 'C+L+S' 'D+EL+L' 'D+EL+S' 'D+ER+L'
 'D+ER+S' 'D+L+S' 'EL+L+S' 'ER+L+S']
[1400    0 2100  700 2800 4900 3500 5600 4200 6300 7000 7700]
[[0 0 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 0]
 [0 1 0 0 1 0 1 0]
 [0 1 0 1 0 0 1 0]
 [0 1 0 1 1 0 0 0]
 [0 1 1 0 0 0 1 0]
 [0 1 1 0 1 0 0 0]
 [1 0 0 0 1 0 1 0]
 [1 0 0 1 0 0 1 0]
 [1 0 0 1 1 0 0 0]
 [1 0 1 0 0 0 1 0]
 [1 0 1 0 1 0 0 0]]
[7700 7000 6300 4200 5600 3500 4900 2800  700 2100    0 1400]


In [8]:
oh_dict = {}
for index in label_indices:
    oh_dict[three_label[index]] = three_label_oh[index]
print(oh_dict)

{'C+EL+L': array([1, 0, 1, 0, 1, 0, 0, 0], dtype=int32), 'C+EL+S': array([1, 0, 1, 0, 0, 0, 1, 0], dtype=int32), 'C+ER+L': array([1, 0, 0, 1, 1, 0, 0, 0], dtype=int32), 'C+ER+S': array([1, 0, 0, 1, 0, 0, 1, 0], dtype=int32), 'C+L+S': array([1, 0, 0, 0, 1, 0, 1, 0], dtype=int32), 'D+EL+L': array([0, 1, 1, 0, 1, 0, 0, 0], dtype=int32), 'D+EL+S': array([0, 1, 1, 0, 0, 0, 1, 0], dtype=int32), 'D+ER+L': array([0, 1, 0, 1, 1, 0, 0, 0], dtype=int32), 'D+ER+S': array([0, 1, 0, 1, 0, 0, 1, 0], dtype=int32), 'D+L+S': array([0, 1, 0, 0, 1, 0, 1, 0], dtype=int32), 'EL+L+S': array([0, 0, 1, 0, 1, 0, 1, 0], dtype=int32), 'ER+L+S': array([0, 0, 0, 1, 1, 0, 1, 0], dtype=int32)}


## 加载模型和更新过的语义

In [9]:
import pickle
model = torch.load('model_saved_pseudo/train_single_two.pth')
with open('updated_semantic_1_2_3/updated_single_dict.pkl', 'rb') as file:
    single_defect_att = pickle.load(file)
with open('updated_semantic_1_2_3/updated_two_dict.pkl', 'rb') as file:
    two_defect_att = pickle.load(file)
with open('updated_semantic_1_2_3/updated_three_dict.pkl', 'rb') as file:
    three_defect_att = pickle.load(file)

In [10]:
three_att = []
for l in three_label:
    three_att.append(three_defect_att[l])
three_att_tensor = torch.tensor(np.array(three_att), dtype=torch.float32)
print(three_att_tensor.shape)

torch.Size([8400, 20])


In [11]:
three_dataset = MyDataSet(three_wm_tensor, three_att_tensor)
three_loader = DataLoader(three_dataset, batch_size=32, shuffle=False)

# 定义预测标签的函数

In [12]:
from py_file.func_Test import Test_Func
# 需要的函数都已经集成在了Test_Func里
func = Test_Func()

# 开始打伪标签

In [13]:
real_label, predict_label = func.get_label(model, three_loader, three_defect_att, 'cos')

In [14]:
real_label == three_label

True

In [15]:
three_label = np.array(three_label)
predict_label = np.array(predict_label)
print(three_label.shape, predict_label.shape)

(8400,) (8400,)


In [16]:
predict_three_label_oh = []
for l in predict_label:
    predict_three_label_oh.append(oh_dict[l])
    
predict_three_label_oh = np.array(predict_three_label_oh)
print(predict_three_label_oh.shape)

(8400, 8)


# 保存打了伪标签的三故障样本

In [17]:
three_ori_wm = np.array(three_ori_wm)
three_wm = np.array(three_wm)
three_label_oh = np.array(three_label_oh)
print(three_ori_wm.shape)
print(three_wm.shape)
print(three_label_oh.shape)

(8400, 52, 52)
(8400, 52, 52)
(8400, 8)


In [18]:
np.savez('data_fake_label/three_fake_label_WM.npz', original_wm = three_ori_wm, label_one_hot = predict_three_label_oh,
 denoise_wm = three_wm, label_name = predict_label)